In [1]:
import pandas as pd
import numpy as np
import sys,os

pd.options.display.max_rows = 10

# Ch10. 데이터 집계와 그룹 연산

## 10.1. GroupBy 메카닉  
분리 - 적용 - 결합 (split-apply-combine)

In [2]:
df = pd.DataFrame({
    'key1':['a','a','b','b','a'],
    'key2':['one','two','one','two','one'],
    'data1':np.random.randn(5),
    'data2':np.random.randn(5),
})
df

,key1,key2,data1,data2
0,a,one,1.769397,0.358055
1,a,two,1.666798,-0.805869
2,b,one,0.599894,0.212237
3,b,two,-0.853862,-0.939432
4,a,one,2.033130,-1.975787


위 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구하기  
groupby 메서드를 호출하면됨

In [3]:
df['data1'].groupby(df['key1'])
grouped = df['data1'].groupby(df['key1'])

GroupBy 객체는 key1으로 참조되는 중간값에 대한것 외에는 아무것도 계산되지 않은 객체임  
그룹연산에 필요한 모든 정보를 포함하고 있어, 각 그룹에 연산을 적용할수 있게함

In [4]:
grouped.sum()
grouped.mean()

key1
a    5.469325
b   -0.253968
Name: data1, dtype: float64

key1
a    1.823108
b   -0.126984
Name: data1, dtype: float64

데이터가 그룹색인에 따라 수집되고 key1컬럼의 유니크한 값으로 색인되는 새로운 Series 객체가 생성됨

여러개의 배열을 넘기면 계층 색인을 가지는 Series가 나옴

In [5]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     1.901264
      two     1.666798
b     one     0.599894
      two    -0.853862
Name: data1, dtype: float64

groupby에 넘기는 객체는 길이만 같으면 어떤것도 상관없음

In [6]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005    1.666798
            2006    0.599894
Ohio        2005    0.457768
            2006    2.033130
Name: data1, dtype: float64

한 그룹으로 묶을 정보를 같은 DataFrame안에서 찾을 경우 컬럼이름을 넘겨서 사용할수 있음

In [7]:
df.groupby('key1').mean()
'''key2의 경우 숫자데이터가 아니기 때문에 (성가신컬럼,nuisance column)결과에서 제외됨'''
df.groupby(['key1','key2']).mean()

,data1,data2
key1,,
a,1.823108,-0.807867
b,-0.126984,-0.363598


'key2의 경우 숫자데이터가 아니기 때문에 (성가신컬럼,nuisance column)결과에서 제외됨'

data1     data2
key1 key2                    
a    one   1.901264 -0.808866
     two   1.666798 -0.805869
b    one   0.599894  0.212237
     two  -0.853862 -0.939432

GroupBy 메서드 중 size메서드는 그룹의 크기를 돌려주므로 유용함

In [8]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹간 순회하기  
groupby 객체는 이터레이션을 지원함  
그룹이름과 그에 따른 데이터 묶음을 반환

In [9]:
for name,data in df.groupby('key1'):
    print(name)
    print(data)
    print('--------------')

a
  key1 key2     data1     data2
0    a  one  1.769397  0.358055
1    a  two  1.666798 -0.805869
4    a  one  2.033130 -1.975787
--------------
b
  key1 key2     data1     data2
2    b  one  0.599894  0.212237
3    b  two -0.853862 -0.939432
--------------


In [10]:
#색인이 여러개일 경우 튜플의 첫번째 원소가 색인값이됨
for k,group in df.groupby(['key1','key2']):
    print(k)
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.769397  0.358055
4    a  one  2.033130 -1.975787
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.666798 -0.805869
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.599894  0.212237
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.853862 -0.939432


In [11]:
#쪼갠데이터를 사전에 넣어쓰기
pieces = dict(list(df.groupby('key1')))
pieces
pieces['a']

{'a':   key1 key2     data1     data2
 0    a  one  1.769397  0.358055
 1    a  two  1.666798 -0.805869
 4    a  one  2.033130 -1.975787, 'b':   key1 key2     data1     data2
 2    b  one  0.599894  0.212237
 3    b  two -0.853862 -0.939432}

,key1,key2,data1,data2
0,a,one,1.769397,0.358055
1,a,two,1.666798,-0.805869
4,a,one,2.033130,-1.975787


기본적으로 axis=0 으로 그룹을 만들지만 다른축도 가능함,  
데이터 타입에 따라 쪼개기

In [12]:
df.dtypes
grouped = df.groupby(df.dtypes,axis=1)
for k,data in grouped:
    print(k,'\n',data)

key1      object
key2      object
data1    float64
data2    float64
dtype: object

float64 
       data1     data2
0  1.769397  0.358055
1  1.666798 -0.805869
2  0.599894  0.212237
3 -0.853862 -0.939432
4  2.033130 -1.975787
object 
   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [13]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [14]:
#신택틱 슈거
df.groupby('key1')['data1']
df.groupby('key2')[['data2']]

array를 넘기면 DF로, 단일값을 넘기면 series

In [15]:
df.groupby(['key1','key2'])[['data1']].mean()
df.groupby(['key1','key2'])['data1'].mean()

data1
key1 key2          
a    one   1.901264
     two   1.666798
b    one   0.599894
     two  -0.853862

key1  key2
a     one     1.901264
      two     1.666798
b     one     0.599894
      two    -0.853862
Name: data1, dtype: float64

### 10.1.3. 사전과 Series에서 그루핑하기

In [16]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = list('abcde'),
                     index = ['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.278828,-0.088485,-1.542517,1.842788,0.587227
Steve,0.986386,-0.721362,0.009484,0.678175,-0.395297
Wes,-0.016554,NaN,NaN,0.472947,-0.284946
Jim,-1.652252,-0.157155,-0.157544,-0.179744,-0.350348
Travis,-0.384617,1.521593,-0.840092,0.711647,0.379887


컬럼에 매핑되는 새로운 그룹이 있고, 그룹별로 더하기

In [17]:
mapping={
    'a':'red',
    'b':'red',
    'c':'blue',
    'd':'blue',
    'e':'red',
    'f':'orange'
}

by_col = people.groupby(mapping,axis=1)
by_col.sum()

,blue,red
Joe,0.300270,0.777570
Steve,0.687659,-0.130273
Wes,0.472947,-0.301501
Jim,-0.337288,-2.159755
Travis,-0.128445,1.516863


In [18]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series,axis=1).count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 함수로 그루핑하기  
사전이나 Series를 이용하는것 보다 파이썬 함수를 이용하는것이 조금더 일반적임  
넘긴 함수는 색인값 하나마다 한번씩 호출됨

이름이 같은 놈끼리 묶으려면..

In [19]:
a = people.groupby(len,axis=0)
[t[0] for t in a]
a.sum()

[3, 5, 6]

,a,b,c,d,e
3,-1.389979,-0.245640,-1.700062,2.135991,-0.048067
5,0.986386,-0.721362,0.009484,0.678175,-0.395297
6,-0.384617,1.521593,-0.840092,0.711647,0.379887


내부적으로 모두 배열로 변환되므로 함수와 다른것을 섞어도 문제가 안됨

In [20]:
key_list = ['one','one','one','two','two']

people.groupby([len,key_list]).mean()

a         b         c         d         e
3 one  0.131137 -0.088485 -1.542517  1.157867  0.151140
  two -1.652252 -0.157155 -0.157544 -0.179744 -0.350348
5 one  0.986386 -0.721362  0.009484  0.678175 -0.395297
6 two -0.384617  1.521593 -0.840092  0.711647  0.379887

### 10.1.5 색인 단계로 그루핑하기  
계층색인에서 하나를 선택하기 위한 기능

In [21]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                   [1,3,5,1,3]],names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)
hier_df
hier_df.groupby(level='cty',axis=1).count()

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.064989 -1.636154 -1.091396 -1.446188  0.888247
1     -0.344163 -0.471375  0.551963 -0.147881  0.453480
2     -0.327596 -1.676407 -0.973909  0.420082  0.107618
3      0.009404 -0.661922  1.580168 -0.412938  1.447888

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2. 데이터 집계  
배열로부터 스칼라값을 만들어 내는 모든 데이터 변환 작업  
- count,sum,mean,median,std,var,min,max,prod,first,last 등등

GroupBy 메서드는 아니지만 Series메서드인 quantile등도 사용가능

In [22]:
df.groupby('key1')['data1'].quantile(0.9)

key1
a    1.980383
b    0.454519
Name: data1, dtype: float64

직접 만든 집계함수를 사용할수도 있음  
- aggregate나 agg 메서드에 함수를 넘기면됨

In [23]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby('key1').agg(peak_to_peak)

,data1,data2
key1,,
a,0.366332,2.333842
b,1.453757,1.151669


In [24]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.823108  0.188980  1.666798  1.718098  1.769397  1.901264   
b      2.0 -0.126984  1.027961 -0.853862 -0.490423 -0.126984  0.236455   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     2.033130   3.0 -0.807867  1.166922 -1.975787 -1.390828 -0.805869   
b     0.599894   2.0 -0.363598  0.814353 -0.939432 -0.651515 -0.363598   

                          
           75%       max  
key1                      
a    -0.223907  0.358055  
b    -0.075681  0.212237

사용자 정의 함수는 일반적으로 GroupBy함수보다 매우 느린데, 중간 데이터를 행성하는 과정에서 호출이나 정렬같은 오버헤드가 발생하기 때문

### 10.2.1. 컬럼에 여러가지 함수 적용하기

In [25]:
tips = pd.read_csv('Datas/tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [26]:
tips.aggregate(np.mean)

total_bill    19.785943
tip            2.998279
size           2.569672
tip_pct        0.160803
dtype: float64

모든 컬럼을 집계하는것은 mean등의 메서드를 사용하거나 원하는 함수에 aggregate를 사용하면되지만  
컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나, 여러개의 함수를 한번에 적용하려면 다음과 같이..